In [2]:
import tatqa_utils
import pandas as pd

In [6]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')
ari = []
for _, item in devdf.iterrows():
    for q in item['questions']:
          if q['answer_type'] == 'arithmetic' and 'table' in q['answer_from']: 
              ari.append({'question':q, 'table': item['table'], 'paragraphs': item['paragraphs']})



In [3]:
import pandas as pd
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2

In [15]:
import os
with open('dataset_raw/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [16]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_community.cache import SQLiteCache

llm = ChatOpenAI(temperature=0)
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [17]:
import re 
def remove_thousand_separators(text):
    # This regular expression matches numbers with commas as thousand separators
    return re.sub(r'(?<=\d),(?=\d{3})', '', text)

In [30]:
def gen_code(llm, question, table):    
    prompt = f"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.\nQuestion: {question} \n Table: {table}"
    res = llm.invoke(prompt)
    code = res.content.replace('```python','').replace('```','')
    return (prompt, code)

def exec_code(code, table):  
    try: 
        loc = locals()   
        if not "run()" in code:
            exec(code + f"\nr = run({table})\n", globals(), loc)
        else: 
            exec(code + "\nr = run()\n", globals(), loc)
        return loc['r']
    except Exception as e:
            s = '[Error]'+ str(e)
            print(s)
            return (s,'')

item = ar[0]
table = item['table']['table']
print(table)

#table = f"{table}"
#table = remove_thousand_separators(table)
#table = tab
#print(table)
question = item['question']['question']
print(question)
( prompt, code) = gen_code(llm, question, table)
res = exec_code(code, table)
print(code)
print(res, item['question']['answer'])

[['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]
What is the change in Other in 2019 from 2018?
def run():
    other_2019 = float(table[3][1])
    other_2018 = float(table[3][2])
    
    change = other_2019 - other_2018
    
    return (change, '')
(-12.600000000000001, '') -12.6


In [24]:
table

[['', '', 'Years Ended September 30,', ''],
 ['', '2019', '2018', '2017'],
 ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'],
 ['Other', '44.1', '56.7', '70.8'],
 ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]